In [1]:
# 回顾点积注意力的写法
# 1. score = QKT / sqrt(d)
# 2. attention_weights = softmax + mask
# 3. dropout(attention_weights) @ value

In [2]:
# 在实践中，当给定相同的查询、键和值的集合时,我们希望模型可以基于相同的注意力机制学习到不同的行为
# 然后将不同的行为作为知识组合起来， 捕获序列内各种范围的依赖关系.例如，短距离依赖和长距离依赖关系）。 
# 因此，允许注意力机制组合使用查询、键和值的不同 子空间表示（representation subspaces）可能是有益的。

# 我们可以用独立学习得到的h组不同的 线性投影（linear projections）来变换查询、键和值。
# 变换后的查询、键和值将并行地送到注意力汇聚中。 
# 最后，将这h个注意力汇聚的输出拼接在一起， 并且通过另一个可以学习的线性投影进行变换， 以产生最终输出。

# 简单来说，就是QKV，投影到不同方向，每个方向有一个head

# 每一个注意力头： hi = f(Wiq, Wik, Wiv)
# 最后连接并输出 output = Wo[h1,h2,h3,...,hh]T

In [3]:
import math
import torch
from torch import nn
from d2l import torch as d2l
# transpose 可以看成 premute 的特例

In [4]:
# 为了避免符号冲突，这里用p代表维度，而不是d
# 设定 pq = pk = pv = po/h , 工程上给一个总维度 / head数 = 每个head的维度
# 下面的实现中，po是通过num_hiddens指定的
# 我们为了复用写过的attention结构，把(B,L,num_hidden)变为(B*h,L,num_hidden/h)

In [8]:
#@save
class MultiHeadAttention(nn.Module):
    def __init__(self, key_size, query_size, value_size,
                num_hiddens, num_heads, dropout, bias=False, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        # q,k,v shape: (B, L, num_hidden)
        # 所有的QKV形状，都是查询的个数，或者是‘键值对’个数，我统一表示为L
        # traspose_qkv 是在把 B,L,num_hiddens -> B,L,num_hiddens/h,h -> B*h,L,num_hiddens
        # transpose_output 是逆向操作
        # def transpose_qkv(X, num_heads)
        queries = transpose_qkv(self.W_q(queries), num_heads)
        keys = transpose_qkv(self.W_k(keys), num_heads)
        values = transpose_qkv(self.W_q(values), num_heads)

        if valid_lens is not None:
            # batch -> B*h,所以valid_lens需要拷贝同等数量
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads,dim=0
            )

        # def forward(self, queries, keys, values, valid_lens=None)
        output = self.attention(queries, keys, values, valid_lens)

        # output_concat的形状:(batch_size，查询的个数，num_hiddens)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)

In [9]:
#@save
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads，
    # num_hiddens/num_heads)
    X = X.reshape(X.shape[0], X.shape[1], num_heads, -1)

    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)

    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数,
    # num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])


#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

In [10]:
# 下面使用键和值相同的小例子来测试我们编写的MultiHeadAttention类。 
# 多头注意力输出的形状是（batch_size，num_queries，num_hiddens）。

num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [12]:
batch_size, num_queries = 2, 4
num_kvpairs, valid_lens =  6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])